In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, metrics
from tqdm import tqdm_notebook as tqdm
import random
from sklearn.dummy import DummyClassifier
from scipy import stats
import math
from skimage import morphology, img_as_bool

In [2]:
path_to_data = "../data/train_images.npy"
path_to_labels = "../data/train_labels.csv"
path_to_test = "../data/test_images.npy"

In [3]:
def plot_images(list_of_images, max_col = 4):
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0], cmap="gray"); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(20, max_col * r))
        for i, (img,name) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title(str(name))
            ax.axis('off')
            ax.imshow(img, cmap="gray")

In [4]:
def get_data(path_x, path_y):
    data = np.load(path_x, encoding = 'bytes')
    labels_df = pd.read_csv(path_y)
    labels_df.Category = pd.Categorical(labels_df.Category)
    y = labels_df.Category.cat.codes.values
    X = np.array(data[:, 1])
    for c, i in enumerate(X):
        ret,thresh_img = cv2.threshold(i,127,255,cv2.THRESH_BINARY)
        X[c] = thresh_img
    return X, y

In [5]:
X_noisy, y = get_data(path_to_data, path_to_labels)
print("Shape of training dataset:", len(X_noisy))
X_test = list(np.load(path_to_test, encoding = 'bytes')[:, 1])
print("Shape of testing dataset:", len(X_test))
unique_classes = set(y)
print("Number of classes:", len(unique_classes))

Shape of training dataset: 10000
Shape of testing dataset: 10000
Number of classes: 31


In [24]:
from scipy import ndimage

def remove_noise(filledImg, min_size = 500):
    blobs, min_val = ndimage.label(filledImg)
    
    for i in range(10,60):
        clean_img = morphology.remove_small_objects(img_as_bool(filledImg), i)
        blobs, number_of_blobs = ndimage.label(clean_img)
        if number_of_blobs < min_val:
            min_val = number_of_blobs
            if min_val == 1:
                return clean_img
    return clean_img

In [25]:
X = []
for c, img in tqdm(enumerate(X_noisy), total = len(X_noisy)):
    img = img.reshape(100, 100)
    X.append(remove_noise(img, 100).flatten())
    

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:137: UserWarning: Possible sign loss when converting negative image of type float64 to positive image of type bool.
  .format(dtypeobj_in, dtypeobj_out))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to bool
  .format(dtypeobj_in, dtypeobj_out))


In [37]:
for c, img in tqdm(enumerate(X)):
    X[c] = np.array(img)
X = np.array(X)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 31
epochs = 12

# input image dimensions
img_rows, img_cols = 100, 100

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = (X[7000:],y[7000:]), (X[:3000],y[:3000]) 
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (3000, 100, 100, 1)
3000 train samples
3000 test samples
Train on 3000 samples, validate on 3000 samples
Epoch 1/12
 512/3000 [====>.........................] - ETA: 3:23 - loss: 3.4311 - acc: 0.0469